<a href="https://colab.research.google.com/github/DanielDialektico/Machine-Learning/blob/main/Notebooks/Aprendizaje%20No%20Supervisado/Agrupaci%C3%B3n/LDA_Modelado_de_T%C3%B3picos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este espacio de trabajo contiene la aplicación de un algoritmo de aprendizaje no supervisado de procesamiento de lenguaje natural a un conjunto de datos de reseñas de clientes de una tienda departamental.

El modelo utilizado fue el LDA ()Latent Dirichlet Allocation), el cual se utiliza para modelado de tópicos, es decir, agrupación de términos en categorías (permite superposición de términos, es decir, cada término puede estar asociado a más de una clase).


# Carga de datos

En el siguiente fragmento de código se instalan y cargan las librerías necesarias.

In [ ]:
!python3 -m spacy download es_core_news_sm
!pip install pyLDAvis
!pip install pyLDAvis.gensim
import pandas as pd
import numpy as np
import re
import spacy
import gensim
from gensim import corpora
import pandas as pd
import nltk
from nltk import FreqDist
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import es_core_news_sm
from nltk.tokenize import word_tokenize
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
nltk.download('punkt')

#Se ignoran las alertas.
warnings.filterwarnings('ignore')
%matplotlib inline

En el siguiente bloque se carga y filtra el conjunto de datos, el cual se obtuvo scrapeando las reseñas de Google Maps utilizando una API:

In [ ]:
#Se carga el conjunto de datos.
reviews = pd.read_csv('https://raw.githubusercontent.com/DanielDialektico/Machine-Learning/main/Conjuntos%20de%20datos/Rese%C3%B1as.csv')
reviews = reviews["Opinión"].dropna().reset_index(drop = True)

# Exploración de datos

Se procede a graficar las 20 palabras más frecuentes.

In [ ]:
# Función para graficar términos más frecuentes
def freq_words(x, terms = 30):
  all_words = ' '.join([text for text in x])
  all_words = all_words.split()

  fdist = FreqDist(all_words)
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})

  # selecting top 20 most frequent words
  d = words_df.nlargest(columns="count", n = terms)
  plt.figure(figsize=(20,5))
  ax = sns.barplot(data=d, x= "word", y = "count")
  ax.set(ylabel = 'Conteo')
  ax.set(xlabel = 'Término')
  plt.xticks(rotation=45, ha="right")
  plt.show()

freq_words(reviews)

Dado que las palabras más frecuentes no aportan información relevante sobre las opiniones de los clientes, se procede a preprocesar el conjunto de datos.

# Preprocesamiento

En la siguiente celda se preprocesa el conjunto de datos aplicando los siguientes ajustes:



*   Se tokenizan las frases.
*   Se convierten todas las letras a minúsculas.
*   Se quitan caracteres no alfanuméricos.
*   Se eliminan stop words.
*   Se remueven adjetivos.
*   Se lematizan los textos.
*   Se remueven los acentos ortográficos.

Al final, se vuelven a graficar los términos más frecuentes.

In [ ]:
nlp = es_core_news_sm.load()

def remove_w(data_set):
  words = ["google", "traducción", "jeje", "gustar", "despota", "tener"]
  for i in words:
    data_set = data_set.str.replace(i, "")
  return data_set

#reviews = remove_w(reviews)

drop_index = []

for i in range(len(reviews)):
  text = nlp(reviews.iloc[i])
  words = [t.orth_ for t in text if not t.is_punct | t.is_stop]
  lexical_tokens = nlp(' '.join([t.lower() for t in words if len(t) > 3 and t.isalpha()]).replace("tender ","tienda "))
  lemmas_no_pron = ' '.join([tok.lemma_ for tok in lexical_tokens if tok.pos_ != 'PRON' and tok.pos_ != 'ADJ'])
  reviews.iat[i] = lemmas_no_pron
  lemmas_as_word = ' '.join(lemmas_no_pron)
  if lemmas_as_word == '':
    drop_index.append(i)

reviews = reviews.drop(drop_index)

reviews = remove_w(reviews)
reviews = reviews.str.replace("tender", "tienda")
reviews = reviews.str.replace("personar", "personal")
reviews = reviews.str.replace("preciar", "precio")
reviews = reviews.str.replace("mesar", "meses")


def remove_accent(data_set):
  accent = ["á","é","í","ó","ú"]
  drop_accent = ["a", "e", "i", "o", "u"]
  for i in range(len(accent)):
    data_set = data_set.str.replace(accent[i], drop_accent[i])
  return data_set

reviews = remove_accent(reviews)

reviews = reviews.apply(lambda x: word_tokenize(x))

reviews_list = reviews.apply(lambda x: ' '.join([j for j in x]))
reviews_list = reviews_list.values.tolist()
freq_words(reviews_list, 35)

Ahora que se ha preprocesado el conjunto de datos, se aplica el algoritmo de aprendizaje no supervisado para la agrupación de términos en categorías.

El siguiente código ajusta el modelo a los datos, e imprime los principales términos que componen a cada una de las 4 categorías generadas.

In [ ]:
dictionary = corpora.Dictionary(reviews)

doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews]

LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=4, random_state = 100,
                chunksize=100, passes=50)

lda_model.print_topics()

Por último, se visualizan los resultados:

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
vis